# Micro exercise 1 (refreshing last lecture)
The code below starts a TCP server on port 1234. The server already answers each connection with a HTTP header. Therefore it is possible to connect to the server with a web browser. Complete the code such that the server reads at most 4096 bytes of the request of the client and echoes it back.

Open http://localhost:1234 in your browser to test the code. You should see the HTTP request of your own browser.

In [ ]:
import asyncio
import time


# Tell on which address and port to listen for connections.
server_addr = 'localhost' # Allow only the local computer to connect.
server_port = 1234

async def main():
    """
    Run a very simple HTTP server that writes back the request of the client.
    """
    
    async def handle_connection(reader: asyncio.StreamReader, writer: asyncio.StreamWriter):
        print("Connection opened!")
        
        # This is a HTTP header for the response.
        # HTTP headers contain meta information such as 
        # wether the request was successful and the name of the server software.
        header = [b"HTTP/1.1 200 OK",
                  b"Server: MicroExerciseEchoServer",
                  b"Connection: Keep-Alive"]
        
        try:
            # STUDENT TASK: Read client request (4096 bytes at most).
            # START SOLUTION (part 1/2)
            
            # END SOLUTION
            
            # Write the HTTP header.
            for line in header:
                writer.write(line)
                writer.write(b'\r\n')
            writer.write(b'\r\n')

            # Write a message.
            writer.write(b'Your HTTP request was:')
            writer.write(b'\r\n\r\n')
            
            # STUDENT TASK: Write back the request.
            # START SOLUTION (part 2/2)
            
            # END SOLUTION

            writer.close()
            await writer.wait_closed()
            print("Connection closed!")
        finally:
            pass
            
    print(f"Starting server on {server_addr}:{server_port}")
    # Create server instance.
    server = await asyncio.start_server(
        handle_connection,
        server_addr,
        server_port
    )

    # Run the server.
    async with server:
        await server.serve_forever()
    
    print("Server terminated.")

await asyncio.wait_for(main(), 60) # Stop the script after at most 60 seconds.
print("Stopped the server.")

# Micro exercise 2
Below is a simple GUI based on Tkinter which exemplifies a typical laboratory scenario where some data is being generated in real time and diplayed on-the-fly, and where some external parameters (for example some voltage) can be set in real-time to influence the experimental setup. In the example, "parameter 1" and "parameter 2" simply change the wavelength of the plotted sine and set an offset on the y axis.

Study the code below, then generate some new data "z" (can be anything) and diplay it in one of the non-utilized quadrants.

In [ ]:
# Observation: Instead of using the main.mainloop() tkinter function, 
# it is possible to use the main.update() function for having
# a better control of when tkinter updates the main window.

import os
import tkinter 
import numpy  
import time
import matplotlib
matplotlib.use('TkAgg') 
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
  
# ===========
# PARAMETERS
#============
default_folder = './'  #'/home/XYZ/'    # Set your default folder here.
default_file_name = 'my_experiment' 

# =========
# FUNCTIONS
#==========
def set_folder():
    path = tkinter.filedialog.askdirectory()
    folder.set(path)

def stop():    
    # Save to file (takes about 100ns in total including opening and closing file):
    output_file = open(folder.get()+file_name.get()+'.txt',  "a")   
    for j in range(len(x)):
        output_file.write(str(x[j])+str("   ")+str(y[j])+"\n")
    output_file.close()
    
    
    main.destroy()  # closing Tk main window.
    try:
        output_file .close()
    except:
        pass  # do nothing
        
# ==============
# GUI definition
# ==============
main = tkinter.Tk()  # main window

# =======================================
# Dividing the space in N vertical frames
# =======================================

# Frame 1 (for inputs)
#=====================
frame_0 = tkinter.Frame(main, relief="sunken", bd=1)
frame_0.pack(side="left", expand=1, fill="both")
# Start button
# ~~~~~~~
def start():
    start_boolean.set(True)
start_boolean = tkinter.BooleanVar(main,  False)
tkinter.Button(frame_0, text='START', font='Calibri 12 bold', foreground='red', width = 20,  height = 3,  wraplength=220, command = start).grid(row=10, column = 0,  sticky='w')
# Stop button
# ~~~~~~~~~~
tkinter.Button(frame_0, text='STOP', font='Calibri 12 bold', foreground='red',               
    width = 20,  height = 3,  wraplength=160, command = stop).grid(row=10, column = 1,  sticky='w')
# File name
# ~~~~~~~~~
tkinter.Label(frame_0, text='File name:', font="Calibri 10 bold").grid(row=0, column=0, sticky='w')
file_name = tkinter.StringVar(main,  value = default_file_name)
tkinter.Entry(frame_0, width=60,  textvariable=file_name, font="Calibri 10").grid(row=0, column=1, sticky='w')
# Output folder
# ~~~~~~~~~~~~~
tkinter.Label(frame_0, text='Folder:', font="Calibri 10 bold").grid(row=1, column=0, sticky='w') 
folder = tkinter.StringVar(main, value=default_folder)
tkinter.Entry(frame_0, width=60, textvariable=folder, font="Calibri 10").grid(row=1, column=1, sticky='w') 

# Parameter 1
# ~~~~~~~~~~~
tkinter.Label(frame_0, text='Parameter 1', font="Calibri 10 bold").grid(row=2, column=0, sticky='w') 
parameter_1 = tkinter.DoubleVar(main,  "3")
tkinter.Spinbox(frame_0, from_=0, to=10, increment = 0.1, textvariable=parameter_1,   font="Calibri 10 bold").grid(row=2, column=1, sticky='w') 
# Parameter 2
# ~~~~~~~~~~~
tkinter.Label(frame_0, text='Parameter 2', font="Calibri 10 bold").grid(row=3, column=0, sticky='w') 
parameter_2 = tkinter.DoubleVar(main,  "0.1")
tkinter.Spinbox(frame_0, from_=0, to=10, increment = 0.1, textvariable=parameter_2,   font="Calibri 10 bold").grid(row=3, column=1, sticky='w') 


# Frame 1 (for output)
#=====================
frame_1 = tkinter.Frame(main, relief="sunken", bd=1)
frame_1.pack(side="left", expand=1, fill="both")

# Frame 2 (for plots)
# ===================
frame_2 = tkinter.Frame(main, relief="sunken", bd=1)
current_frame = frame_2
current_frame.pack(side="left", expand=1, fill="both")
figure_for_subplots = Figure(figsize=(25, 15), dpi=80)
# Placing the figure into the tkinter window:
canvas_for_subplots = FigureCanvasTkAgg(figure_for_subplots, master=current_frame)
canvas_for_subplots.get_tk_widget().pack()  #canvas._tkcanvas.pack(side=tkinter.TOP, fill=tkinter.BOTH, expand=1)
plot1 = figure_for_subplots.add_subplot(221) # Format: add_subplot( # of colums, # of rows, current subplot). Index increases as normal text (left to right, then new line)
plot2 = figure_for_subplots.add_subplot(222)
plot3 = figure_for_subplots.add_subplot(223)
plot4 = figure_for_subplots.add_subplot(224)


# Experiment
#===========


# Initialization
x = numpy.zeros(0, dtype = float) # Initialize an empty array of zero length and float type
y = numpy.zeros(0, dtype = float) # Initialize an empty array of zero length and float type
counter = 0 # It represents the number of measurements.

# Endless loop represeting the running experiment.
while True:    
    # Do nothing untill Start button is pressed:
    while not(start_boolean.get()):
        main.update()  # Do nothing.
            
    # Create some data for the plot:
    y_offset = parameter_1.get()
    x_scale = parameter_2.get()
    x_new = counter  
    y_new = numpy.sin(x_scale/10*x_new) + y_offset
    # Add new measurement point to the data array.
    x = numpy.hstack([x, x_new ])
    y = numpy.hstack([y, y_new ])  
    
    counter = counter + 1
                        
    # Refresh plot1
    current_plot = plot1
    current_plot.plot(x , y, color="blue", marker='.',linestyle='')
    current_plot.set_title("Some title A")
    current_plot.set_xlabel("x [a.u.]")
    current_plot.set_ylabel("y [a.u.]")
    current_plot.set_xlim(xmin=min(x), xmax=max(x))
    current_plot.grid(True)
    canvas_for_subplots.draw()      
    main.update()   # Update the tkinter main window.
    current_plot.clear()
                        
    # Refresh plot4 (same as plot1 but without .clear())
    current_plot = plot4
    current_plot.plot(x , y, color="red", marker='o',linestyle='')
    current_plot.set_title("Some title B")
    current_plot.set_xlabel("x [a.u.]")
    current_plot.set_ylabel("y [a.u.]")
    current_plot.set_xlim(xmin=min(x), xmax=max(x))
    current_plot.grid(True)
    canvas_for_subplots.draw()      
    main.update()   # Update the tkinter main window.
    #current_plot.clear()




# Exercise 1
The following code represents the hypothetical scenario of controlling an "Agilent sourcemeter type B2912A" using the LXI Ethernet interface and the SCPI commands.

This example assumes that the instrument is reachable at the internet address defined by the host name "ief-lab-b2912a-2.ee.ethz.ch". Since this machine is not available today, **the code will produce an error** if executed.

For keeping order when dealing with several instruments, different instruments are defined by objects defined in individual files placed in the `Exercise_11/Instruments/` folder.

Study the code below as well as the file `Intruments/Agilent_B2912A.py`. In particular notice the use of the socket module and of the TCP socket which was introduced in the previous lecture.

Finally, write a *single* Python cell/file (which does not use/import any instrument objects) with as few lines of code as you can which turns on the output of the sourcemeter. Hint: analyse the methods in the file Agilent_B2912A.py and copy-paste/modify just the minimum amount of lines in your solution.

In [ ]:
import time
# Import intruments from the "Instruments" folder:
#from Instruments.Agilent_8163B import Agilent_8163B
#from Instruments.Keysight_N7744A import Keysight_N7744A
from Instruments.Agilent_B2912A import Agilent_B2912A
#from Instruments.Agilent_8510C import Agilent_8510C


# Open the sourcemeter:
print('Opening Agilent sourcemeter...')
source_meter = Agilent_B2912A(host = 'ief-lab-b2912a-2.ee.ethz.ch')  # XXX SET CORRECT ADDRESS
source_meter.open()
# Set voltage and compliance current:
source_meter.set_voltage(voltage = 2, channel = 1)
source_meter.set_compliance_current(current = 5E-3, channel = 1) # Set protection current in Ampere
# Turn voltage on:
source_meter.send_string(':OUTP ON')  # This is an example where the SCPI command is run directly from the main code.
time.sleep(0.2) # Sleep for 0.2 s.
# Read and printing data:
data = source_meter.spot_measurement(channel=1)
time.sleep(0.1)
# Turn voltage off:
source_meter.send_string(':OUTP OFF')
# Delete the source_meter object (this will also close the socket):
del source_meter  
# Printing meaured values:
print('Measured voltage: {} V, measured current: {} A, set voltage: {} V'.format(data[0], data[1], data[2]))

In [ ]:
## BEGIN SOLUTION




## END SOLUTION

# Exercise 2
Modify the file `Instruments/Agilent_B2912A.py` and add a method of your choice. The simplest example could be a method called `idn()` which runs the indentity command. For help look at the provided programming manual.

# Exericise 3 (optional)
The instrument "HP 8153A" is an instrument which has no Ethernet adapter, but only a GPIB port. This intrument can connected to network using the LXI Ethernet adapter described in the lecture and it is possible to communicate with it using the file `Instruments/HP_8153A.py`.

Study the file `Instruments/HP_8153A.py` and the manual `Manuals/Prologix_GPIB_Ethernet.pdf`, then describe which part of the code is special (when compared to the source-meter above) for getting the LXI adapter to work.

Write your answer here (double-click to edit): ...


# Uploading solutions
Before the end of the class at about 16:00, please "push" your solutions. 

Please do so even if you have not solved all problems: additional
uploads can be made in the following days. Instructions are below.

### If git is available on your system (preferred option)
Add, commit and push your changes to the remote server:

`git add -A`

`git commit -m 'My solutions to Lecture XX'`

`git push origin master`

### If git is **not** available on your system
This is **not** the favourite solution and it should be avoided whenever possible.

Upload your Lecture_XX folder (containing the Exercise file) to the polybox https://polybox.ethz.ch and share the folder with luca.alloatti@ief.ee.ethz.ch, thomas.kramer@ief.ee.ethz.ch, and raphael.schwanninger@ief.ee.ethz.ch . To share the folder go on https://polybox.ethz.ch , then on the right of the folder there is a graph with one vertex connecting to two other vertices: click on it and then type the three emails.